In [8]:
import hashlib
from enum import Enum


class PaymentMethod(Enum):
    NONE = 0
    CRYPTO = 1
    CASH = 2
    UPI = 3
    DIGITAL = 4


class RideSharing:
    def __init__(self, driver_address):
        self.driver = driver_address
        self.rider = None
        self.fare = 0
        self.ride_completed = False
        self.payment_method = PaymentMethod.NONE
        self.payment_received = False
        self.original_hash = None

    def get_ride_data(self):
        return {
            "rider": self.rider,
            "driver": self.driver,
            "fare": self.fare,
            "payment_method": self.payment_method.name,
            "ride_completed": self.ride_completed,
            "payment_received": self.payment_received
        }

    def generate_hash(self, data: dict):
        data_str = str(sorted(data.items()))
        return hashlib.sha256(data_str.encode()).hexdigest()

    def request_ride(self, rider_name, rider_address, fare, payment_type):
        self.rider = f"{rider_name} ({rider_address})"
        self.fare = fare
        self.payment_method = payment_type
        self.ride_completed = False
        self.payment_received = False

        # Generate and store original hash
        self.original_hash = self.generate_hash(self.get_ride_data())

        print(f"\n{self.rider} calls requestRide()")
        print(f" → Fare = ₹{self.fare}")
        print(f" → Payment method = {self.payment_method.name}")

        print("\nSmart contract records:")
        print(f" - rider = {self.rider}")
        print(f" - driver = {self.driver}")
        print(f" - fare = {self.fare}")
        print(f" - paymentMethod = {self.payment_method.name}")
        print(f" - rideStatus = \"In Progress\"")

    def complete_ride(self, driver_address, payment_received):
        if driver_address != self.driver:
            print("[Error] Only the assigned driver can complete the ride.")
            return

        # Tamper check
        current_hash = self.generate_hash(self.get_ride_data())
        if current_hash != self.original_hash:
            print("⚠️ Tampering Detected! Ride data was changed after creation.")
            return
        else:
            print("✅ Data Integrity Verified.")

        if not payment_received:
            print("[Error] Payment not received yet.")
            return

        self.ride_completed = True
        self.payment_received = True

        print("\nLater:")
        print(f"Driver ({self.driver}) confirms payment received via {self.payment_method.name}")

        print("\nSmart contract updates:")
        print(f" - rideStatus = \"Completed\"")
        print(f" - paymentConfirmed = {str(self.payment_received).lower()}")


# ---------------------------
# 🚀 Main Program Starts Here
# ---------------------------
def main():
    driver_address = "0xDriverXYZ"
    ride = RideSharing(driver_address)

    # 🚗 Get user input
    rider_name = input("Enter Rider Name: ").strip()

    print("\nSelect Payment Method:")
    print("1. CRYPTO")
    print("2. CASH")
    print("3. UPI")
    print("4. DIGITAL")

    payment_choice = input("Enter your choice (1-4): ").strip()
    method_map = {
        "1": PaymentMethod.CRYPTO,
        "2": PaymentMethod.CASH,
        "3": PaymentMethod.UPI,
        "4": PaymentMethod.DIGITAL
    }

    if payment_choice not in method_map:
        print("Invalid payment method selected. Exiting.")
        return

    payment_method = method_map[payment_choice]

    # 🔐 Wallet address required only for crypto
    if payment_method == PaymentMethod.CRYPTO:
        rider_address = input("Enter Rider Wallet Address (required for crypto): ").strip()
        if not rider_address:
            print("❌ Wallet address is required for crypto payments. Exiting.")
            return
    else:
        rider_address = input("Enter Rider Wallet Address (optional): ").strip()
        if not rider_address:
            rider_address = "N/A"

    try:
        fare = int(input("Enter Fare (₹): ").strip())
    except ValueError:
        print("Invalid fare amount. Exiting.")
        return

    # Request ride
    ride.request_ride(rider_name, rider_address, fare, payment_method)

    # Optional: Simulate tampering
    # ride.fare = 999  # Uncomment to test tampering detection

    # Confirm payment
    confirm = input("\nHas the driver received payment? (yes/no): ").strip().lower()
    if confirm == "yes":
        ride.complete_ride(driver_address, payment_received=True)
    else:
        print("\n[Info] Ride is still in progress. Waiting for payment confirmation...")


# Run the program
if __name__ == "__main__":
    main()


Enter Rider Name: hafeeza

Select Payment Method:
1. CRYPTO
2. CASH
3. UPI
4. DIGITAL
Enter your choice (1-4): 2
Enter Rider Wallet Address (optional): 
Enter Fare (₹): 300

hafeeza (N/A) calls requestRide()
 → Fare = ₹300
 → Payment method = CASH

Smart contract records:
 - rider = hafeeza (N/A)
 - driver = 0xDriverXYZ
 - fare = 300
 - paymentMethod = CASH
 - rideStatus = "In Progress"

Has the driver received payment? (yes/no): no

[Info] Ride is still in progress. Waiting for payment confirmation...
